In [213]:
import scipy.io as io
import mat73
import pandas as pd
import numpy as np
from ipywidgets import widgets
from IPython.display import clear_output
from arcgis.gis import GIS
from arcgis.raster.functions import *
from arcgis.mapping.ogc import GeoJSONLayer
import time

from osm_runner import Runner  # pip install osm-runner
import pandas as pd
from arcgis.geocoding import geocode
from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor, use_proximity
from arcgis.geoenrichment import enrich
from arcgis import dissolve_boundaries
from arcgis.geometry import project, Point, Polyline, Polygon, Geometry, lengths, areas_and_lengths

In [220]:
#Log In
gis = GIS('https://foretifydev.maps.arcgis.com/home/index.html', 'analyst2022', 'ForetifyTillIDie2022!!')

#map
map1 = gis.map()
map1

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
study_area_dict = {'California':'http://services.arcgis.com/PpEMp4p6SBYbe0zW/arcgis/rest/services/California_Counties/FeatureServer/0',
                   'Montana':'http://services.arcgis.com/iTQUx5ZpNUh47Geb/arcgis/rest/services/Montana_Mask/FeatureServer/0',
                   'Oregon':'https://services.arcgis.com/uUvqNMGPm7axC2dD/arcgis/rest/services/Oregon_Boundary_generalized/FeatureServer/0',
                   'Texas':'http://services2.arcgis.com/5MVN2jsqIrNZD4tP/arcgis/rest/services/Texas_Outline/FeatureServer/0'}

study_areas = ['California',
               'Montana',
               'Nevada',
               'Oregon',
               'Texas']

country = widgets.Dropdown(
    options=study_areas,
    value='California',
    description='Region to Process:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        location = geocode(str(country.value) + ', USA')[0]
        map1.extent = location['extent']


country.observe(on_change)

display(country)

Dropdown(description='Region to Process:', options=('California', 'Montana', 'Nevada', 'Oregon', 'Texas'), val…

In [4]:
data_dictionary = {
'EPTL'   : 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Transmission_Lines_gdb/FeatureServer/0',
'ES'     : 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Electric_Substations_1/FeatureServer/0',
'CA'     : 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Control_Area/FeatureServer/0',
'PTR'    : 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Public_Transit_Routes/FeatureServer/0',
'NGP'    : 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Natural_Gas_Liquid_Pipelines/FeatureServer/0',
'ERST'   : 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/ArcGIS/rest/services/Retail_Service_Territories/FeatureServer/0',
'NSLPCA' : '',
'PADUS'  : '',
'FHZ'    : 'https://hazards.fema.gov/gis/nfhl/rest/services/FIRMette/NFHLREST_FIRMette/MapServer/20'
}

In [6]:
def feature_server_to_df(feature_layer = None):
    '''
    Grab feature layer from server and turn into DataFrame.
    
    Choose one of the following feature_layer options (name (type) - option), 
    ### denotes unavailability for server:
    
    Electric Power Transmission Lines (str) - EPTL
    
    Electric Substations (str) - ES
    
    Control Areas (str) - CA
    
    Public Transit Routes (str) - PTR
    
    Natural Gas Pipelines (str)  - NGP
    
    Electric Retail Service Territories (str) - ERST
    
    ### National State Local and Private Conservation Areas (str) - NSLPCA 
    
    ### Protected Database of the US (str) - PADUS
    
    Flood Hazard Zones (str) - FHZ
    
    '''
    
    if feature_layer == 'EPTL':
        
        grab_data = FeatureLayer(data_dictionary[feature_layer])
        
        data_df = GeoAccessor.from_layer(grab_data)
                        
        return data_df
        
    elif feature_layer == 'ES':
        
        grab_data = FeatureLayer(data_dictionary[feature_layer])
        
        data_df = GeoAccessor.from_layer(grab_data)
                
        return data_df
        
    elif feature_layer == 'CA':
        
        grab_data = FeatureLayer(data_dictionary[feature_layer])
        
        data_df = GeoAccessor.from_layer(grab_data)
                
        return data_df
        
    elif feature_layer == 'PTR':
        
        grab_data = FeatureLayer(data_dictionary[feature_layer])
        
        data_df = GeoAccessor.from_layer(grab_data)
                
        return data_df
    
    elif feature_layer == 'NGP':
        
        grab_data = FeatureLayer(data_dictionary[feature_layer])
        
        data_df = GeoAccessor.from_layer(grab_data)
                
        return data_df

    elif feature_layer == 'ERST':
        
        grab_data = FeatureLayer(data_dictionary[feature_layer])
        
        data_df = GeoAccessor.from_layer(grab_data)
                
        return data_df
        
    elif feature_layer == 'NSLPCA':
        
        print('Currently unavailable, please choose another feature')
        
        return None

    elif feature_layer == 'PADUS':

        print('Currently unavailable, please choose another feature')
        
        return None

    elif feature_layer == 'FHZ':

        grab_data = FeatureLayer(data_dictionary[feature_layer])

        data_df = GeoAccessor.from_layer(grab_data)
        
        return data_df

In [8]:
ERST_df = feature_server_to_df('ERST')

In [202]:
def get_geometry(dataframe, state = None, idx = None, end_idx = None):
    '''
    To return all, by state, or by index geometries from dataframe.
    from a dataframe and display map centered on geometry.
    
    state (str) - Capitalized abbreviation of state
    
    idx (int) - Index of DataFrame
    
    map_object (map variable) - Variable name of map
    '''
    
    dataframe = dataframe.loc[dataframe['COUNTRY'] == 'USA'].reset_index(drop=True)
    
    map_object = gis.map()
    
    if state == None and idx == None:
        
        all_raw_geometry = dataframe['SHAPE']
        
        for i in range(len(dataframe)):
            
            map_object.draw(all_raw_geometry[i])
            
        return all_raw_geometry, map_object
            
    if state != None:
        
        dataframe = dataframe.loc[dataframe['STATE'] == state].reset_index(drop=True)
        
        state_geometry = dataframe['SHAPE']
        
        map_object = gis.map(f'{state}, USA')
        
        for j in range(len(dataframe)):
            
            map_object.draw(state_geometry[j])
            
        return state_geometry, map_object
    
    if idx != None and end_idx == None:
        
        dataframe = dataframe.iloc[[idx]].reset_index(drop=True)
        
        idx_geometry = dataframe['SHAPE']
        
        state = dataframe['STATE'][idx]
    
        map_object = gis.map(f'{state}, USA')
        
        for j in range(len(dataframe)):
            
            map_object.draw(idx_geometry[j])
            
        return idx_geometry, map_object
    
    if idx != None and end_idx != None:
        
        dataframe=dataframe.iloc[idx:end_idx].reset_index(drop=True)
        
        idx_to_idx_geometry = dataframe['SHAPE']
        
        for k in range(len(dataframe)):
            
            map_object.draw(idx_to_idx_geometry[k])
            
        return idx_to_idx_geometry, map_object
        

In [206]:
rawgeom, testmap1 = get_geometry(ERST_df, idx = 0, end_idx = 2)

In [211]:
poly = rawgeom[0]

In [212]:
def create_buffer(geometry, distance):
    pass

In [221]:
use_proximity.create_buffers(poly, distances=[50], units = 'Miles')

Invalid value for parameter inputLayer - Details : Invalid JSON for GPFeatureRecordSetLayer or GPRecordSet datatype 
Failed.


Exception: Job failed.